<a href="https://colab.research.google.com/github/ayeonggg/DaconPrac/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import pandas as pd
import numpy as np
import os
import seaborn as sns
from sklearn.metrics import accuracy_score
import xgboost as xgb
xgb_model = xgb.XGBRegressor()
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.metrics import auc, precision_recall_curve, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import pickle
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib as mpl
%matplotlib inline

In [2]:
# load data
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

In [3]:
train_data

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [4]:
test_data

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1_20220825 00,1,20220825 00,23.5,0.0,2.2,72
1,1_20220825 01,1,20220825 01,23.0,0.0,0.9,72
2,1_20220825 02,1,20220825 02,22.7,0.0,1.5,75
3,1_20220825 03,1,20220825 03,22.1,0.0,1.3,78
4,1_20220825 04,1,20220825 04,21.8,0.0,1.0,77
...,...,...,...,...,...,...,...
16795,100_20220831 19,100,20220831 19,22.5,0.0,0.9,84
16796,100_20220831 20,100,20220831 20,20.7,0.0,0.4,95
16797,100_20220831 21,100,20220831 21,20.2,0.0,0.4,98
16798,100_20220831 22,100,20220831 22,20.1,0.0,1.1,97


In [5]:
# 결측치 처리
train_data['강수량(mm)'].fillna(0, inplace=True)
train_data['풍속(m/s)'].fillna(train_data['풍속(m/s)'].mean(), inplace=True)
train_data['습도(%)'].fillna(train_data['습도(%)'].mean(), inplace=True)
train_data['일조(hr)'].fillna(0, inplace=True)
train_data['일사(MJ/m2)'].fillna(0, inplace=True)
test_data['강수량(mm)'].fillna(0, inplace=True)
test_data['일조(hr)'] = 0
test_data['일사(MJ/m2)'] = 0

In [6]:
# 날짜 및 시간 특성 파생
train_data['일시'] = pd.to_datetime(train_data['일시'], format='%Y%m%d %H')
test_data['일시'] = pd.to_datetime(test_data['일시'], format='%Y%m%d %H')
train_data['연'] = train_data['일시'].dt.year
train_data['월'] = train_data['일시'].dt.month
train_data['일'] = train_data['일시'].dt.day
train_data['시간'] = train_data['일시'].dt.hour
test_data['연'] = test_data['일시'].dt.year
test_data['월'] = test_data['일시'].dt.month
test_data['일'] = test_data['일시'].dt.day
test_data['시간'] = test_data['일시'].dt.hour

# 필요하지 않은 컬럼 제거
train_data.drop(columns=['num_date_time', '일시'], inplace=True)
test_data.drop(columns=['num_date_time', '일시'], inplace=True)


In [8]:
# 특성과 라벨 분리
X_train = train_data.drop(columns=['전력소비량(kWh)'])
y_train = train_data['전력소비량(kWh)']

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_data)

In [10]:
# 데이터 분할 (훈련 및 검증 세트)
seed=7
test_size=0.33
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

In [11]:
# 하이퍼파라미터 그리드 정의
param_grid= {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [5000],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

In [12]:
# XGBoost 모델 생성
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

In [28]:
from sklearn.model_selection import RandomizedSearchCV
import random
# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid,
                                   n_iter=10, cv=3, n_jobs=1, verbose=2, scoring='neg_root_mean_squared_error', random_state=42)

# 랜덤 서치 수행
random_search.fit(X_train_split, y_train_split)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END colsample_bytree=0.9, learning_rate=0.1, max_depth=7, n_estimators=5000, subsample=0.9; total time= 6.6min
[CV] END colsample_bytree=0.9, learning_rate=0.1, max_depth=7, n_estimators=5000, subsample=0.9; total time= 6.5min
[CV] END colsample_bytree=0.9, learning_rate=0.1, max_depth=7, n_estimators=5000, subsample=0.9; total time= 6.5min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=5000, subsample=0.9; total time= 2.1min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=5000, subsample=0.9; total time= 2.1min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, n_estimators=5000, subsample=0.9; total time= 2.1min
[CV] END colsample_bytree=0.9, learning_rate=0.05, max_depth=5, n_estimators=5000, subsample=0.7; total time= 4.8min
[CV] END colsample_bytree=0.9, learning_rate=0.05, max_depth=5, n_estimators=5000, subsample=0.7; total time= 4.5min
[CV] END 

RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=42, ...),
                   n_jobs=1,
                   param_distributions={'colsample_bytree': [0.7, 0.9],
                                        'learning_rate': [0.01, 0.05, 0.1],
                                        'max_depth': [3, 5, 7],
                                        'n_estimators': [5000],
                                        'subsample': [0.7, 0.9]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [29]:
# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", random_search.best_params_)

Best Hyperparameters: {'subsample': 0.9, 'n_estimators': 5000, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.9}


In [30]:
# 검증 세트에서 최적 모델의 성능 평가
y_val_pred_best = random_search.best_estimator_.predict(X_val_split)
val_rmse_best = np.sqrt(mean_squared_error(y_val_split, y_val_pred_best))
print("Validation RMSE with Best Model:", val_rmse_best)

Validation RMSE with Best Model: 287.1954117697001


In [31]:
# test 데이터에 대한 예측
y_test_pred_best = random_search.best_estimator_.predict(X_test_scaled)
test_predictions_best = pd.DataFrame({'전력소비량(kWh)': y_test_pred_best})
test_predictions_best.to_csv('test_predictions_best.csv', index=False)